In [7]:
from eval import *
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics.cluster import *
from sklearn.datasets import fetch_20newsgroups
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
torch.cuda.set_device(6)

In [8]:
# get data 
newsgroups_train = fetch_20newsgroups(data_home="~/workspace/scikit_learn_data", subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(data_home="~/workspace/scikit_learn_data", subset='test', remove=('headers', 'footers', 'quotes'))

text, labels = newsgroups_train.data, newsgroups_train.target
test_batch_size=1000
size = len(labels)
print("Loaded dataset {} with total lines: {}".format("20 NEWS", size))

top_level_labels = np.copy(labels)
top_categories = dict((name, i) for (i, name) in enumerate(set(map(lambda x: x.split('.')[0], newsgroups_test.target_names))))
#print(top_categories)
for i, name in enumerate(newsgroups_test.target_names):
    #print(i, name)
    top = name.split('.')[0]
    top_level_labels[labels == i ] = top_categories[top]

print(top_level_labels[:15])
print(labels[:15])

Loaded dataset 20 NEWS with total lines: 11314
[0 4 4 4 6 5 6 4 4 4 0 5 4 6 3]
[ 7  4  4  1 14 16 13  3  2  4  8 19  4 14  6]


In [12]:
#load taboola model and encode 
taboola_model_dir = "/home/diq/all/bert_expe/tabooola_gensim_model"
d2v = Doc2Vec.load("{}/{}".format(taboola_model_dir, "gensim_doc2vec.model"))

d2v_features = np.vstack([d2v.infer_vector(simple_preprocess(doc)) for doc in text])
print(d2v_features.shape)

#assert np.array_equal(d2v_features[0, :],  d2v.infer_vector(simple_preprocess(text[0])))

2019-07-19 14:59:11 INFO     loading Doc2Vec object from /home/diq/all/bert_expe/tabooola_gensim_model/gensim_doc2vec.model
2019-07-19 14:59:12 INFO     loading vocabulary recursively from /home/diq/all/bert_expe/tabooola_gensim_model/gensim_doc2vec.model.vocabulary.* with mmap=None
2019-07-19 14:59:12 INFO     loading trainables recursively from /home/diq/all/bert_expe/tabooola_gensim_model/gensim_doc2vec.model.trainables.* with mmap=None
2019-07-19 14:59:12 INFO     loading syn1neg from /home/diq/all/bert_expe/tabooola_gensim_model/gensim_doc2vec.model.trainables.syn1neg.npy with mmap=None
2019-07-19 14:59:12 INFO     loading wv recursively from /home/diq/all/bert_expe/tabooola_gensim_model/gensim_doc2vec.model.wv.* with mmap=None
2019-07-19 14:59:12 INFO     loading vectors from /home/diq/all/bert_expe/tabooola_gensim_model/gensim_doc2vec.model.wv.vectors.npy with mmap=None
2019-07-19 14:59:13 INFO     loading docvecs recursively from /home/diq/all/bert_expe/tabooola_gensim_model/ge

(11314, 600)


In [13]:
#load qt model
checkpoint_dir = '/home/jcjessecai/workspace/taboola/quickthoughts/checkpoints'
with open("{}/config.json".format(checkpoint_dir)) as fp:
    CONFIG = json.load(fp)

WV_MODEL = api.load(CONFIG['embedding'])
qt = QuickThoughts(WV_MODEL, hidden_size=CONFIG['hidden_size'])
trained_params = torch.load("{}/checkpoint_latest.pth".format(checkpoint_dir))
qt.load_state_dict(trained_params['state_dict'])
qt = qt.cuda()
qt.eval()
print("Restored successfully from {}".format(checkpoint_dir))

2019-07-19 14:59:37 INFO     loading projection weights from /home/jcjessecai/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
2019-07-19 15:01:38 INFO     loaded (400000, 300) matrix from /home/jcjessecai/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
/home/jcjessecai/workspace/miniconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Restored successfully from /home/jcjessecai/workspace/taboola/quickthoughts/checkpoints


In [14]:
#encode data
def make_batch(j):
    """Processes one test batch of the test datset"""
    stop_idx = min(size, j+test_batch_size)
    batch_text, batch_labels  = text[j:stop_idx], labels[j:stop_idx]
    data = list(map(lambda x: torch.LongTensor(prepare_sequence(x, WV_MODEL.vocab, no_zeros=True)), batch_text))
    for i in data:
        if len(i) == 0:
            print(i)
            input()
    packed = safe_pack_sequence(data).cuda()
    return qt(packed).cpu().detach().numpy()

feature_list = [make_batch(i) for i in range(0, size, test_batch_size)]
print("Processing {:5d} batches of size {:5d}".format(len(feature_list), test_batch_size))
qt_features = np.concatenate(feature_list)
print("Test feature matrix of shape: {}".format(qt_features.shape))

Processing    12 batches of size  1000
Test feature matrix of shape: (11314, 1000)


In [15]:
#first we compare embedding performance by fitting binary classifier on top
s=1
X_train, X_test, y_train, y_test = train_test_split(d2v_features, labels)
clf = LogisticRegression(solver='sag', multi_class='multinomial')
clf.fit(X_train, y_train)
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("Fit logistic model on d2v with s: {:3d} and train acc: {:.2%} test acc: {:.2%}".format(s, train_acc, test_acc))

Fit logistic model on d2v with s:   1 and train acc: 79.84% test acc: 53.23%


/home/jcjessecai/workspace/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(qt_features, labels)
clf = LogisticRegression(solver='sag', multi_class='multinomial')
clf.fit(X_train, y_train)
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("Fit logistic model on qt with s: {:3d} and train acc: {:.2%} test acc: {:.2%}".format(s, train_acc, test_acc))

Fit logistic model on qt with s:   1 and train acc: 93.68% test acc: 47.01%


/home/jcjessecai/workspace/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [17]:
qt_predicted  = KMeans(n_clusters=7, n_jobs=20).fit_predict(qt_features)
d2v_predicted  = KMeans(n_clusters=7, n_jobs=20).fit_predict(d2v_features)

In [18]:
print(adjusted_rand_score(top_level_labels, qt_predicted))
print(adjusted_rand_score(top_level_labels, d2v_predicted))

print(adjusted_mutual_info_score(top_level_labels, qt_predicted))
print(adjusted_mutual_info_score(top_level_labels, d2v_predicted))

0.12372624528161412
0.10892941037885245
0.16328080524560495
0.1614446379480039


/home/jcjessecai/workspace/miniconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

feature_embedded = TSNE(n_components=2, verbose=1).fit_transform(features[:5000])

fig, axs = plt.subplots(4, 5, figsize=(20, 15))

fixed_xlim, fixed_ylim = (-85.6129455010451, 110.84618372125996), (-90.00594782812799, 80.691349744632)

for i in range(20):
    selected = feature_embedded[labels[:5000] == i]
    ax = axs[i//5, i%5]
    ax.scatter(selected[:, 0], selected[:, 1])
    ax.set_ylim(fixed_ylim)
    ax.set_xlim(fixed_xlim)
    
plt.show()

fig, axs = plt.subplots(4, 5, figsize=(20, 15))

fixed_xlim, fixed_ylim = (-85.6129455010451, 110.84618372125996), (-90.00594782812799, 80.691349744632)

for i in range(20):
    selected = feature_embedded[predicted_spectral[:5000] == i]
    ax = axs[i//5, i%5]
    ax.scatter(selected[:, 0], selected[:, 1])
    ax.set_ylim(fixed_ylim)
    ax.set_xlim(fixed_xlim)
    
plt.show()

fig, axs = plt.subplots(4, 5, figsize=(20, 15))

fixed_xlim, fixed_ylim = (-85.6129455010451, 110.84618372125996), (-90.00594782812799, 80.691349744632)

for i in range(20):
    selected = feature_embedded[predicted_kmeans[:5000] == i]
    ax = axs[i//5, i%5]
    ax.scatter(selected[:, 0], selected[:, 1])
    ax.set_ylim(fixed_ylim)
    ax.set_xlim(fixed_xlim)
    
plt.show()